<a href="https://colab.research.google.com/github/Lairai7/Coursera_Capstone/blob/master/Data_science_Capstone_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Introduction**



Your Fairytale Wedding' is a company based in Toronto which plans larger-than-life, dream weddings for people. The wedding day is a magical one for any couple, signifying new beginnings of a beautiful future together and 'Your Fairytale Wedding' strives to make it as memorable as possible. To achieve this important task, it is required to provide its staff an exhaustive list of all the different places in a given vicinity that come into picture while organizing a wedding so that they can then proceed further, personalizing the wedding experience according to the needs and wants of their clients.

**Data Description**

I shall be using my Foursquare Developer credentials to access Foursquare location data for this Capstone project. As we discussed in the ‘Introduction’ section, ‘Your Fairytale Wedding’ has to furnish its staff with all the details of places relevant to planning a wedding. Also, since ‘Your Fairytale Wedding’ believes in innovation and doing things differently, they provide their clients with the option to choose the menu for their wedding buffet from their favorite restaurants, rather than sticking to the cookie-cutter approach of a single catering service. Thus, coming back to our discussion of incorporating Foursquare API in this project, our search queries will include Bakery, bridal shops, gift shops,cosmetics and clothing stores, restaurants,hotels, banquet halls or event spaces and churches. Data wrangling or preprocessing will then be performed on these datasets followed by clustering as well as a map representation using Folium library. 

**Importing the necessary libraries** 

In [2]:
import requests # to handle requests
import pandas as pd # for data analsysis
import numpy as np # to handle data in a vectorized manner

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
#tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library for map representation
print("Libraries imported!")

Libraries imported!


**Defining Foursquare credentials** 

In [3]:
ClIENT_ID = 'PUQQSNWDSUKXX20LQP1ISLWSRGGTKACD3IEQMGVQGFVCPXRJ' # your Foursquare ID
ClIENT_SECRET = 'L3BPVYMKFNEL4GE5PIXL255Z540TKZ1AXKKHH5KSURY3MXH2' # your Foursquare Secret
VERSION = '20180604'
LIMIT =30
print('Your credentials:')
print('Foursquare_ID: ' + ClIENT_ID)
print('Foursquare_Secret:' + ClIENT_SECRET)

Your credentials:
Foursquare_ID: PUQQSNWDSUKXX20LQP1ISLWSRGGTKACD3IEQMGVQGFVCPXRJ
Foursquare_Secret:L3BPVYMKFNEL4GE5PIXL255Z540TKZ1AXKKHH5KSURY3MXH2


**Selecting the city and getting its coordinates**


In [4]:
#selecting the city and getting its latitude and longitude

city = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)


43.653963 -79.387207


**Hotels and Event Spaces- time for a grand celebration!**

In [5]:
# search for hotels
search_query = 'Hotel'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PUQQSNWDSUKXX20LQP1ISLWSRGGTKACD3IEQMGVQGFVCPXRJ&client_secret=L3BPVYMKFNEL4GE5PIXL255Z540TKZ1AXKKHH5KSURY3MXH2&ll=43.653963,-79.387207&v=20180604&query=Hotel&radius=500&limit=30'

In [0]:
# Sending the GET Request and examining the results
results = requests.get(url).json()

In [7]:
# assigning relevant part of JSON to venues
venues = results['response']['venues']

# tranforming venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.neighborhood
0,4ab2d511f964a5209b6c20e3,Sheraton Centre Toronto Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1583463651,False,123 Queen Street West,at York St.,43.650594,-79.384530,"[{'label': 'display', 'lat': 43.6505944, 'lng'...",432,M5H 2M9,CA,Toronto,ON,Canada,"[123 Queen Street West (at York St.), Toronto ...",NaN,NaN
1,4b68aed1f964a520de862be3,The Rex Hotel Jazz & Blues Bar,"[{'id': '4bf58dd8d48988d1e7931735', 'name': 'J...",v-1583463651,False,194 Queen St W,Queen & St. Patrick,43.650505,-79.388577,"[{'label': 'display', 'lat': 43.65050475544005...",400,M5V 1Z1,CA,Toronto,ON,Canada,"[194 Queen St W (Queen & St. Patrick), Toronto...",62225795,NaN
2,4f343a31e4b0230a3b337a90,VFM Test Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1583463651,False,123 Test Drive,at somewhere St,43.658434,-79.387894,"[{'label': 'display', 'lat': 43.658434, 'lng':...",500,M2M 2M2,CA,Toronto,ON,Canada,"[123 Test Drive (at somewhere St), Toronto ON ...",NaN,NaN
3,5d59d01867339e000897bc88,Sheraton Centre Hotel Club Lounge,"[{'id': '4bf58dd8d48988d121941735', 'name': 'L...",v-1583463651,False,NaN,NaN,43.651063,-79.384527,"[{'label': 'display', 'lat': 43.651063, 'lng':...",388,M5H 2M9,CA,Toronto,ON,Canada,"[Toronto ON M5H 2M9, Canada]",NaN,NaN
4,4f53fb2ee4b036244bea152f,Stathcona Hotel,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1583463651,False,NaN,NaN,43.654947,-79.386359,"[{'label': 'display', 'lat': 43.654947, 'lng':...",129,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN


**Data wrangling**

In [8]:
# keep columns including venue name, and those associated with location only
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_dataframe = dataframe.loc[:,clean_columns]

# function to extract the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_dataframe['categories'] = clean_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_dataframe.columns = [column.split('.')[-1] for column in clean_dataframe.columns]

clean_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,at York St.,43.650594,-79.384530,"[{'label': 'display', 'lat': 43.6505944, 'lng'...",432,M5H 2M9,CA,Toronto,ON,Canada,"[123 Queen Street West (at York St.), Toronto ...",NaN,4ab2d511f964a5209b6c20e3
1,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,Queen & St. Patrick,43.650505,-79.388577,"[{'label': 'display', 'lat': 43.65050475544005...",400,M5V 1Z1,CA,Toronto,ON,Canada,"[194 Queen St W (Queen & St. Patrick), Toronto...",NaN,4b68aed1f964a520de862be3
2,VFM Test Hotel,Hotel,123 Test Drive,at somewhere St,43.658434,-79.387894,"[{'label': 'display', 'lat': 43.658434, 'lng':...",500,M2M 2M2,CA,Toronto,ON,Canada,"[123 Test Drive (at somewhere St), Toronto ON ...",NaN,4f343a31e4b0230a3b337a90
3,Sheraton Centre Hotel Club Lounge,Lounge,NaN,NaN,43.651063,-79.384527,"[{'label': 'display', 'lat': 43.651063, 'lng':...",388,M5H 2M9,CA,Toronto,ON,Canada,"[Toronto ON M5H 2M9, Canada]",NaN,5d59d01867339e000897bc88
4,Stathcona Hotel,Hotel,NaN,NaN,43.654947,-79.386359,"[{'label': 'display', 'lat': 43.654947, 'lng':...",129,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,4f53fb2ee4b036244bea152f


In [14]:
# drop unnecessary columns
clean_dataframe2= clean_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs','neighborhood','id'], axis=1)
clean_dataframe2



,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
2,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
3,Sheraton Centre Hotel Club Lounge,Lounge,NaN,43.651063,-79.384527,M5H 2M9,ON
4,Stathcona Hotel,Hotel,NaN,43.654947,-79.386359,NaN,ON
5,Sheraton Centre Toronto Hotel - Grand Ballroom,Ballroom,123 Queen Street West,43.651200,-79.384520,NaN,ON
6,Be SixFifty Hotel,Hotel,650 Bay Street,43.657120,-79.384560,NaN,ON
7,650 Hotel,Hotel,650 Bay Street,43.657046,-79.384411,NaN,ON
8,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
9,Shangri-La Toronto,Hotel,188 University Ave.,43.649129,-79.386557,M5H 0A3,ON


In [15]:
# drop rows with none values
clean_dataframe3 = clean_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
1,The Rex Hotel Jazz & Blues Bar,Jazz Club,194 Queen St W,43.650505,-79.388577,M5V 1Z1,ON
2,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
8,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
9,Shangri-La Toronto,Hotel,188 University Ave.,43.649129,-79.386557,M5H 0A3,ON
10,Hilton,Hotel,145 Richmond St W,43.649946,-79.385479,M5H 2L2,ON
11,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON
12,89 Chestnut Residence,College Residence Hall,89 Chestnut St.,43.654160,-79.385291,M5G 1R1,ON
14,Fitness Centre,Gym,Sheraton Centre,43.650985,-79.384002,M5H 2M9,ON
15,Lobby Lounge at the Shangri-La Toronto,Lounge,188 University Ave.,43.649155,-79.386546,M5H 0A3,ON


In [16]:
# keep only those rows with the relevant categories-'Hotel' and 'Event space'
array= ['Hotel', 'Event Space']
hotel_dataframe= clean_dataframe3.loc[clean_dataframe3['categories'].isin(array)]
hotel_dataframe

,name,categories,address,lat,lng,postalCode,state
0,Sheraton Centre Toronto Hotel,Hotel,123 Queen Street West,43.650594,-79.384530,M5H 2M9,ON
2,VFM Test Hotel,Hotel,123 Test Drive,43.658434,-79.387894,M2M 2M2,ON
8,DoubleTree by Hilton,Hotel,108 Chestnut Street,43.654608,-79.385942,M5G 1R3,ON
9,Shangri-La Toronto,Hotel,188 University Ave.,43.649129,-79.386557,M5H 0A3,ON
10,Hilton,Hotel,145 Richmond St W,43.649946,-79.385479,M5H 2L2,ON
11,Marriott Downtown at CF Toronto Eaton Centre,Hotel,525 Bay Street,43.654728,-79.382422,M5G 2L2,ON
20,Grand Ballroom,Event Space,123 Queen St. W,43.651217,-79.383771,M5H 2M9,ON


**Bridal shops, gifts and cosmetics, clothing stores- Let's pamper the bride!**

In [18]:
# search for Bridal shops
search_query = 'Bridal shop'
radius = 10000

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PUQQSNWDSUKXX20LQP1ISLWSRGGTKACD3IEQMGVQGFVCPXRJ&client_secret=L3BPVYMKFNEL4GE5PIXL255Z540TKZ1AXKKHH5KSURY3MXH2&ll=43.653963,-79.387207&v=20180604&query=Bridal shop&radius=10000&limit=30'

In [0]:
# Sending the GET Request and examining the results
results2 = requests.get(url).json()


In [20]:
# assigning relevant part of JSON to venues
venues = results2['response']['venues']

# tranform venues into a dataframe
bridalshop_dataframe = json_normalize(venues)
bridalshop_dataframe.head()

,id,name,categories,referralId,hasPerk,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.address,location.postalCode,location.crossStreet,venuePage.id,location.neighborhood
0,4e420754aeb79367c7e3be1e,Queens Bridal Shop Toronto,"[{'id': '4bf58dd8d48988d11a951735', 'name': 'B...",v-1583463807,False,43.648935,-79.383921,"[{'label': 'display', 'lat': 43.648935, 'lng':...",619,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN,NaN,NaN,NaN
1,4b6dbabef964a5204d8a2ce3,Bridal Fashion Fraire,"[{'id': '4bf58dd8d48988d11a951735', 'name': 'B...",v-1583463807,False,43.646984,-79.395803,"[{'label': 'display', 'lat': 43.6469839165044,...",1040,CA,Toronto,ON,Canada,"[123 Spadina Ave., Toronto ON, Canada]",123 Spadina Ave.,NaN,NaN,NaN,NaN
2,4d13cab06d103704a15d1bbd,Gift Shop,"[{'id': '4bf58dd8d48988d128951735', 'name': 'G...",v-1583463807,False,43.654151,-79.392729,"[{'label': 'display', 'lat': 43.65415122702497...",445,CA,Toronto,ON,Canada,"[317 Dundas Street W, Toronto ON M5T 1G4, Canada]",317 Dundas Street W,M5T 1G4,NaN,NaN,NaN
3,5c2346cb029a55002c80c510,F5 Shop,"[{'id': '52f2ab2ebcbc57f1066b8b36', 'name': 'I...",v-1583463807,False,43.654211,-79.385715,"[{'label': 'display', 'lat': 43.65421113388417...",123,CA,East York,ON,Canada,"[1559 Bayview Ave, East York ON M4G 3B5, Canada]",1559 Bayview Ave,M4G 3B5,NaN,NaN,NaN
4,59e636b7840fc2686ffb51f1,Square Shop!,"[{'id': '52f2ab2ebcbc57f1066b8b3d', 'name': 'P...",v-1583463807,False,43.650440,-79.388885,"[{'label': 'display', 'lat': 43.65044, 'lng': ...",414,CA,Toronto,ON,Canada,"[202 Queen Street West (St Patrick), Toronto O...",202 Queen Street West,M5V 1Z2,St Patrick,NaN,NaN


**Data wrangling-cleaning bridal shop dataframe**

In [21]:
#Cleaning bridal shop dataframe
#keep only columns that include venue name, and anything that is associated with location
bridalshop_clean_columns = ['name', 'categories'] + [col for col in bridalshop_dataframe.columns if col.startswith('location.')]+ ['id']
clean_bridalshop_dataframe = bridalshop_dataframe.loc[:,bridalshop_clean_columns]

# function to extract the category of the venue
def get_category_type(row):
    try:
        categories_list1 = row['categories']
    except:
        categories_list1 = row['venue.categories']
        
    if len(categories_list1) == 0:
        return None
    else:
        return categories_list1[0]['name']

# filter the category for each row
clean_bridalshop_dataframe['categories'] = clean_bridalshop_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_bridalshop_dataframe.columns = [column.split('.')[-1] for column in clean_bridalshop_dataframe.columns]

clean_bridalshop_dataframe.head()

,name,categories,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,address,postalCode,crossStreet,neighborhood,id
0,Queens Bridal Shop Toronto,Bridal Shop,43.648935,-79.383921,"[{'label': 'display', 'lat': 43.648935, 'lng':...",619,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",NaN,NaN,NaN,NaN,4e420754aeb79367c7e3be1e
1,Bridal Fashion Fraire,Bridal Shop,43.646984,-79.395803,"[{'label': 'display', 'lat': 43.6469839165044,...",1040,CA,Toronto,ON,Canada,"[123 Spadina Ave., Toronto ON, Canada]",123 Spadina Ave.,NaN,NaN,NaN,4b6dbabef964a5204d8a2ce3
2,Gift Shop,Gift Shop,43.654151,-79.392729,"[{'label': 'display', 'lat': 43.65415122702497...",445,CA,Toronto,ON,Canada,"[317 Dundas Street W, Toronto ON M5T 1G4, Canada]",317 Dundas Street W,M5T 1G4,NaN,NaN,4d13cab06d103704a15d1bbd
3,F5 Shop,IT Services,43.654211,-79.385715,"[{'label': 'display', 'lat': 43.65421113388417...",123,CA,East York,ON,Canada,"[1559 Bayview Ave, East York ON M4G 3B5, Canada]",1559 Bayview Ave,M4G 3B5,NaN,NaN,5c2346cb029a55002c80c510
4,Square Shop!,Pop-Up Shop,43.650440,-79.388885,"[{'label': 'display', 'lat': 43.65044, 'lng': ...",414,CA,Toronto,ON,Canada,"[202 Queen Street West (St Patrick), Toronto O...",202 Queen Street West,M5V 1Z2,St Patrick,NaN,59e636b7840fc2686ffb51f1


In [22]:
# drop unnecessary columns
clean_bridalshop_dataframe2= clean_bridalshop_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',\
                                        'labeledLatLngs', 'id','neighborhood','postalCode'], axis=1)
clean_bridalshop_dataframe2

,name,categories,lat,lng,state,address
0,Queens Bridal Shop Toronto,Bridal Shop,43.648935,-79.383921,ON,NaN
1,Bridal Fashion Fraire,Bridal Shop,43.646984,-79.395803,ON,123 Spadina Ave.
2,Gift Shop,Gift Shop,43.654151,-79.392729,ON,317 Dundas Street W
3,F5 Shop,IT Services,43.654211,-79.385715,ON,1559 Bayview Ave
4,Square Shop!,Pop-Up Shop,43.650440,-79.388885,ON,202 Queen Street West
5,Friendly Stranger - Cannabis Culture Shop,Smoke Shop,43.650387,-79.388523,ON,241 Queen St. W
6,The Hippie Grow Shop,Garden,43.652688,-79.387807,ON,"1370 Don Mills Road, 3rd Floor"
7,Sushi Shop,Sushi Restaurant,43.661620,-79.387636,ON,"76 Grenville St,Woman's College Hospital"
8,Jays Shop at Sears,Sporting Goods Shop,43.655823,-79.380491,ON,290 Yonge St.
9,The Body Shop,Cosmetics Shop,43.649086,-79.394368,ON,393 Queen St W


In [23]:
# drop rows with none values
clean_bridalshop_dataframe3 = clean_bridalshop_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_bridalshop_dataframe3

,name,categories,lat,lng,state,address
1,Bridal Fashion Fraire,Bridal Shop,43.646984,-79.395803,ON,123 Spadina Ave.
2,Gift Shop,Gift Shop,43.654151,-79.392729,ON,317 Dundas Street W
3,F5 Shop,IT Services,43.654211,-79.385715,ON,1559 Bayview Ave
4,Square Shop!,Pop-Up Shop,43.650440,-79.388885,ON,202 Queen Street West
5,Friendly Stranger - Cannabis Culture Shop,Smoke Shop,43.650387,-79.388523,ON,241 Queen St. W
6,The Hippie Grow Shop,Garden,43.652688,-79.387807,ON,"1370 Don Mills Road, 3rd Floor"
7,Sushi Shop,Sushi Restaurant,43.661620,-79.387636,ON,"76 Grenville St,Woman's College Hospital"
8,Jays Shop at Sears,Sporting Goods Shop,43.655823,-79.380491,ON,290 Yonge St.
9,The Body Shop,Cosmetics Shop,43.649086,-79.394368,ON,393 Queen St W
11,The Body Shop,Cosmetics Shop,43.646525,-79.380145,ON,"200 Bay Street, Unit 103"


In [24]:
# keep only those rows with the relevant categories-bridal shops, cosmetics and gift shops, clothing stores
array= ['Bridal Shop', 'Gift Shop','Cosmetics Shop','Clothing Store']
bridalshop_dataframe= clean_bridalshop_dataframe3.loc[clean_bridalshop_dataframe3['categories'].isin(array)]
bridalshop_dataframe

,name,categories,lat,lng,state,address
1,Bridal Fashion Fraire,Bridal Shop,43.646984,-79.395803,ON,123 Spadina Ave.
2,Gift Shop,Gift Shop,43.654151,-79.392729,ON,317 Dundas Street W
9,The Body Shop,Cosmetics Shop,43.649086,-79.394368,ON,393 Queen St W
11,The Body Shop,Cosmetics Shop,43.646525,-79.380145,ON,"200 Bay Street, Unit 103"
14,Gift Shop,Gift Shop,43.665595,-79.380148,Ontario,Holiday Inn
23,StyleDemocracy Designer Pop-Up Shop,Clothing Store,43.644685,-79.399359,ON,580 King St W
24,5 Fifty 5 Shop,Gift Shop,43.657254,-79.386692,ON,555 University Ave


**Churches- Exchanging wedding vows...'Till Death Do Us Part!'**

In [25]:
# search for churches
search_query = 'Church'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PUQQSNWDSUKXX20LQP1ISLWSRGGTKACD3IEQMGVQGFVCPXRJ&client_secret=L3BPVYMKFNEL4GE5PIXL255Z540TKZ1AXKKHH5KSURY3MXH2&ll=43.653963,-79.387207&v=20180604&query=Church&radius=500&limit=30'

In [0]:
# Sending the GET Request and examining the results
results = requests.get(url).json()

In [27]:
# assigning relevant part of JSON to venues
venues = results['response']['venues']

# tranforming venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,location.postalCode
0,4e6cfd5318381712da1c9ddf,Our Lady of Mount Carmel Catholic Church 嘉模聖母天主堂,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",v-1583463840,False,202 St. Patrick St.,43.655329,-79.390672,"[{'label': 'display', 'lat': 43.65532892596064...",317,CA,Toronto,ON,Canada,"[202 St. Patrick St., Toronto ON, Canada]",NaN,NaN
1,4b91b671f964a520f3d133e3,St. Patrick's Church,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",v-1583463840,False,141 McCaul St.,43.654551,-79.391359,"[{'label': 'display', 'lat': 43.65455071624035...",340,CA,Toronto,ON,Canada,"[141 McCaul St. (Dundas St.), Toronto ON M5T 1...",Dundas St.,M5T 1W3
2,4dd838e91f6ee146833794a1,St. George The Martyr Anglican Church,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",v-1583463840,False,205 John St,43.651568,-79.391267,"[{'label': 'display', 'lat': 43.65156753541776...",421,CA,Toronto,ON,Canada,"[205 John St (Stephanie St), Toronto ON, Canada]",Stephanie St,NaN
3,4c815c12d34ca143e2b72180,Holy Trinity Russian Orthodox Church,"[{'id': '4bf58dd8d48988d132941735', 'name': 'C...",v-1583463840,False,23 Henry St.,43.657248,-79.393972,"[{'label': 'display', 'lat': 43.65724806332831...",656,CA,Toronto,ON,Canada,"[23 Henry St., Toronto ON, Canada]",NaN,NaN
4,55163055498e69a823177e08,church street toronto,"[{'id': '4bf58dd8d48988d11a941735', 'name': 'O...",v-1583463840,False,NaN,43.652034,-79.387303,"[{'label': 'display', 'lat': 43.65203443955587...",214,CA,NaN,NaN,Canada,[Canada],NaN,NaN


**Data wrangling- Cleaning Church dataframe**

In [28]:
# keep columns including venue name, and those associated with location only
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_church_dataframe = dataframe.loc[:,clean_columns]

# function to extract the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_church_dataframe['categories'] = clean_church_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_church_dataframe.columns = [column.split('.')[-1] for column in clean_church_dataframe.columns]

clean_church_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,id
0,Our Lady of Mount Carmel Catholic Church 嘉模聖母天主堂,Church,202 St. Patrick St.,43.655329,-79.390672,"[{'label': 'display', 'lat': 43.65532892596064...",317,CA,Toronto,ON,Canada,"[202 St. Patrick St., Toronto ON, Canada]",NaN,NaN,4e6cfd5318381712da1c9ddf
1,St. Patrick's Church,Church,141 McCaul St.,43.654551,-79.391359,"[{'label': 'display', 'lat': 43.65455071624035...",340,CA,Toronto,ON,Canada,"[141 McCaul St. (Dundas St.), Toronto ON M5T 1...",Dundas St.,M5T 1W3,4b91b671f964a520f3d133e3
2,St. George The Martyr Anglican Church,Church,205 John St,43.651568,-79.391267,"[{'label': 'display', 'lat': 43.65156753541776...",421,CA,Toronto,ON,Canada,"[205 John St (Stephanie St), Toronto ON, Canada]",Stephanie St,NaN,4dd838e91f6ee146833794a1
3,Holy Trinity Russian Orthodox Church,Church,23 Henry St.,43.657248,-79.393972,"[{'label': 'display', 'lat': 43.65724806332831...",656,CA,Toronto,ON,Canada,"[23 Henry St., Toronto ON, Canada]",NaN,NaN,4c815c12d34ca143e2b72180
4,church street toronto,Other Nightlife,NaN,43.652034,-79.387303,"[{'label': 'display', 'lat': 43.65203443955587...",214,CA,NaN,NaN,Canada,[Canada],NaN,NaN,55163055498e69a823177e08


In [29]:
# drop unnecessary columns
clean_church_dataframe2= clean_church_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',
                                        'labeledLatLngs','id','postalCode'], axis=1)
clean_church_dataframe2

,name,categories,address,lat,lng,state
0,Our Lady of Mount Carmel Catholic Church 嘉模聖母天主堂,Church,202 St. Patrick St.,43.655329,-79.390672,ON
1,St. Patrick's Church,Church,141 McCaul St.,43.654551,-79.391359,ON
2,St. George The Martyr Anglican Church,Church,205 John St,43.651568,-79.391267,ON
3,Holy Trinity Russian Orthodox Church,Church,23 Henry St.,43.657248,-79.393972,ON
4,church street toronto,Other Nightlife,NaN,43.652034,-79.387303,NaN
5,The Church of the Holy Trinity,Church,10 Trinity Sq,43.654580,-79.381705,ON
6,St. Nicholas Catholic church,Church,NaN,43.654872,-79.390857,NaN
7,Chinese Presbyterian Church,Church,177 Beverley St,43.655635,-79.394259,ON
8,Holy Trinity Russian Orthodox Church,Historic Site,NaN,43.657242,-79.394037,NaN
9,Winston Churchill,Sculpture Garden,NaN,43.651019,-79.387193,NaN


In [30]:
# drop rows with none values
clean_church_dataframe3 = clean_church_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_church_dataframe3

,name,categories,address,lat,lng,state
0,Our Lady of Mount Carmel Catholic Church 嘉模聖母天主堂,Church,202 St. Patrick St.,43.655329,-79.390672,ON
1,St. Patrick's Church,Church,141 McCaul St.,43.654551,-79.391359,ON
2,St. George The Martyr Anglican Church,Church,205 John St,43.651568,-79.391267,ON
3,Holy Trinity Russian Orthodox Church,Church,23 Henry St.,43.657248,-79.393972,ON
5,The Church of the Holy Trinity,Church,10 Trinity Sq,43.654580,-79.381705,ON
7,Chinese Presbyterian Church,Church,177 Beverley St,43.655635,-79.394259,ON


*Here as you can see above, after dropping the rows with none values, we are left with our desired category only- Churches, so we were saved the additional operation of removing the irrelevant categories namely 'Sculpture Garden','Other Nightlife' and 'Historic Site'.*

**Bakery- Let's find that perfect wedding cake!**

In [31]:
# search for bakery
search_query = 'Bakery'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'\
.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PUQQSNWDSUKXX20LQP1ISLWSRGGTKACD3IEQMGVQGFVCPXRJ&client_secret=L3BPVYMKFNEL4GE5PIXL255Z540TKZ1AXKKHH5KSURY3MXH2&ll=43.653963,-79.387207&v=20180604&query=Bakery&radius=500&limit=30'

In [0]:
# Sending the GET Request and examining the results
results = requests.get(url).json()

In [33]:
# assigning relevant part of JSON to venues
venues = results['response']['venues']

# tranforming venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.cc,location.city,location.state,location.country,location.formattedAddress,location.postalCode
0,4b9f9b5af964a5206b2e37e3,Bakery 18,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",v-1583463869,False,595 Bay St.,at Dundas St. W,43.656765,-79.383548,"[{'label': 'display', 'lat': 43.65676534941227...",429,CA,Toronto,ON,Canada,"[595 Bay St. (at Dundas St. W), Toronto ON, Ca...",NaN
1,53542f20498eb5c7149ecafd,Bakery & Kaffee Haus,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",v-1583463869,False,238 Queen Street West,NaN,43.650096,-79.390378,"[{'label': 'display', 'lat': 43.6500955307706,...",500,CA,Toronto,ON,Canada,"[238 Queen Street West, Toronto ON M5V 0B5, Ca...",M5V 0B5
2,4eadd3dbe5fad025f84c97f4,Kin-Kin Bakery & Bubble Tea,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",v-1583463869,False,595 Bay St.,NaN,43.656198,-79.382403,"[{'label': 'display', 'lat': 43.65619846393864...",460,CA,Toronto,ON,Canada,"[595 Bay St., Toronto ON M5G 2C2, Canada]",M5G 2C2
3,57462ae4498efddb24245014,Paris Croissant Bakery Cafe,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",v-1583463869,False,595 Bay St,Dundas St West,43.656204,-79.383210,"[{'label': 'display', 'lat': 43.656204, 'lng':...",407,CA,Toronto,ON,Canada,"[595 Bay St (Dundas St West), Toronto ON, Canada]",NaN
4,4afc5b80f964a520e42122e3,Fresh Start Coffee Co.,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",v-1583463869,False,655 Bay St,at Elm St,43.657377,-79.384393,"[{'label': 'display', 'lat': 43.65737716511621...",442,CA,Toronto,ON,Canada,"[655 Bay St (at Elm St), Toronto ON M5G 2K4, C...",M5G 2K4


**Cleaning the Bakery dataframe**

In [34]:
# keep columns including venue name, and those associated with location only
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_bakery_dataframe = dataframe.loc[:,clean_columns]

# function to extract the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_bakery_dataframe['categories'] = clean_bakery_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_bakery_dataframe.columns = [column.split('.')[-1] for column in clean_bakery_dataframe.columns]

clean_bakery_dataframe.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,postalCode,id
0,Bakery 18,Bakery,595 Bay St.,at Dundas St. W,43.656765,-79.383548,"[{'label': 'display', 'lat': 43.65676534941227...",429,CA,Toronto,ON,Canada,"[595 Bay St. (at Dundas St. W), Toronto ON, Ca...",NaN,4b9f9b5af964a5206b2e37e3
1,Bakery & Kaffee Haus,Bakery,238 Queen Street West,NaN,43.650096,-79.390378,"[{'label': 'display', 'lat': 43.6500955307706,...",500,CA,Toronto,ON,Canada,"[238 Queen Street West, Toronto ON M5V 0B5, Ca...",M5V 0B5,53542f20498eb5c7149ecafd
2,Kin-Kin Bakery & Bubble Tea,Bakery,595 Bay St.,NaN,43.656198,-79.382403,"[{'label': 'display', 'lat': 43.65619846393864...",460,CA,Toronto,ON,Canada,"[595 Bay St., Toronto ON M5G 2C2, Canada]",M5G 2C2,4eadd3dbe5fad025f84c97f4
3,Paris Croissant Bakery Cafe,Bakery,595 Bay St,Dundas St West,43.656204,-79.383210,"[{'label': 'display', 'lat': 43.656204, 'lng':...",407,CA,Toronto,ON,Canada,"[595 Bay St (Dundas St West), Toronto ON, Canada]",NaN,57462ae4498efddb24245014
4,Fresh Start Coffee Co.,Café,655 Bay St,at Elm St,43.657377,-79.384393,"[{'label': 'display', 'lat': 43.65737716511621...",442,CA,Toronto,ON,Canada,"[655 Bay St (at Elm St), Toronto ON M5G 2K4, C...",M5G 2K4,4afc5b80f964a520e42122e3


In [35]:
# drop unnecessary columns
clean_bakery_dataframe2= clean_bakery_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',
                                        'labeledLatLngs', 'id','postalCode'], axis=1)
clean_bakery_dataframe2

,name,categories,address,lat,lng,state
0,Bakery 18,Bakery,595 Bay St.,43.656765,-79.383548,ON
1,Bakery & Kaffee Haus,Bakery,238 Queen Street West,43.650096,-79.390378,ON
2,Kin-Kin Bakery & Bubble Tea,Bakery,595 Bay St.,43.656198,-79.382403,ON
3,Paris Croissant Bakery Cafe,Bakery,595 Bay St,43.656204,-79.383210,ON
4,Fresh Start Coffee Co.,Café,655 Bay St,43.657377,-79.384393,ON


In [36]:
# keep only those rows with the relevant category-Bakery
clean_bakery_dataframe3 = clean_bakery_dataframe2[clean_bakery_dataframe2.categories == 'Bakery']
clean_bakery_dataframe3

,name,categories,address,lat,lng,state
0,Bakery 18,Bakery,595 Bay St.,43.656765,-79.383548,ON
1,Bakery & Kaffee Haus,Bakery,238 Queen Street West,43.650096,-79.390378,ON
2,Kin-Kin Bakery & Bubble Tea,Bakery,595 Bay St.,43.656198,-79.382403,ON
3,Paris Croissant Bakery Cafe,Bakery,595 Bay St,43.656204,-79.383210,ON


**Restaurants-The most delicious part,your wedding buffet!**

In [37]:
# search for Restaurants
search_query = 'Restaurant'
radius = 500

# Define the corresponding URL
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(ClIENT_ID, ClIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=PUQQSNWDSUKXX20LQP1ISLWSRGGTKACD3IEQMGVQGFVCPXRJ&client_secret=L3BPVYMKFNEL4GE5PIXL255Z540TKZ1AXKKHH5KSURY3MXH2&ll=43.653963,-79.387207&v=20180604&query=Restaurant&radius=500&limit=30'

In [0]:
# Sending the GET Request and examining the results
results = requests.get(url).json()

In [39]:
# assigning relevant part of JSON to venues
venues = results['response']['venues']

# tranforming venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id
0,4ad4c05ff964a52048f720e3,Hemispheres Restaurant & Bistro,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1583463900,False,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",145,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,NaN
1,5750b013498e755287c6de97,Some Time BBQ Grill Restaurant 碳烤屋,"[{'id': '52af3b773cf9994f4e043c03', 'name': 'S...",v-1583463900,False,988 Baldwin Street,43.655874,-79.393826,"[{'label': 'display', 'lat': 43.655874, 'lng':...",573,NaN,CA,Toronto,ON,Canada,"[988 Baldwin Street, Toronto ON, Canada]",NaN,NaN
2,4b2027b5f964a520f82d24e3,Hong Shing Chinese Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1583463900,False,195 Dundas St W,43.654925,-79.387089,"[{'label': 'display', 'lat': 43.65492521335936...",107,M5G 1C7,CA,Toronto,ON,Canada,"[195 Dundas St W (at University Ave), Toronto ...",at University Ave,60327598
3,4c69740b8d22c9284d42b745,Wah Too Seafood Restaurant,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1583463900,False,56 Centre Ave.,43.654833,-79.387206,"[{'label': 'display', 'lat': 43.65483285234745...",96,M5G 1R5,CA,Toronto,ON,Canada,"[56 Centre Ave., Toronto ON M5G 1R5, Canada]",NaN,NaN
4,4ae4b055f964a520229d21e3,Kyoto House Japanese Restaurant,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1583463900,False,143 Dundas St. West,43.655381,-79.385270,"[{'label': 'display', 'lat': 43.65538110598594...",221,NaN,CA,Toronto,ON,Canada,"[143 Dundas St. West, Toronto ON, Canada]",NaN,NaN


**Cleaning Restaurant dataframe**

In [40]:
# keep columns including venue name, and those associated with location only
clean_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')]+ ['id']
clean_restaurant_dataframe = dataframe.loc[:,clean_columns]

# function to extract the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
clean_restaurant_dataframe['categories'] = clean_restaurant_dataframe.apply(get_category_type, axis=1)

# clean column names by keeping only last term
clean_restaurant_dataframe.columns = [column.split('.')[-1] for column in clean_restaurant_dataframe.columns]

clean_restaurant_dataframe.head()

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,"[{'label': 'display', 'lat': 43.65488413420439...",145,M5G 1R3,CA,Toronto,ON,Canada,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]",NaN,4ad4c05ff964a52048f720e3
1,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,43.655874,-79.393826,"[{'label': 'display', 'lat': 43.655874, 'lng':...",573,NaN,CA,Toronto,ON,Canada,"[988 Baldwin Street, Toronto ON, Canada]",NaN,5750b013498e755287c6de97
2,Hong Shing Chinese Restaurant,Chinese Restaurant,195 Dundas St W,43.654925,-79.387089,"[{'label': 'display', 'lat': 43.65492521335936...",107,M5G 1C7,CA,Toronto,ON,Canada,"[195 Dundas St W (at University Ave), Toronto ...",at University Ave,4b2027b5f964a520f82d24e3
3,Wah Too Seafood Restaurant,Chinese Restaurant,56 Centre Ave.,43.654833,-79.387206,"[{'label': 'display', 'lat': 43.65483285234745...",96,M5G 1R5,CA,Toronto,ON,Canada,"[56 Centre Ave., Toronto ON M5G 1R5, Canada]",NaN,4c69740b8d22c9284d42b745
4,Kyoto House Japanese Restaurant,Sushi Restaurant,143 Dundas St. West,43.655381,-79.385270,"[{'label': 'display', 'lat': 43.65538110598594...",221,NaN,CA,Toronto,ON,Canada,"[143 Dundas St. West, Toronto ON, Canada]",NaN,4ae4b055f964a520229d21e3


In [41]:
# drop unnecessary columns
clean_restaurant_dataframe2= clean_restaurant_dataframe.drop(['cc', 'city', 'country', 'crossStreet', 'distance', 'formattedAddress',
                                        'labeledLatLngs', 'id'], axis=1)
clean_restaurant_dataframe2

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
1,Some Time BBQ Grill Restaurant 碳烤屋,Szechuan Restaurant,988 Baldwin Street,43.655874,-79.393826,NaN,ON
2,Hong Shing Chinese Restaurant,Chinese Restaurant,195 Dundas St W,43.654925,-79.387089,M5G 1C7,ON
3,Wah Too Seafood Restaurant,Chinese Restaurant,56 Centre Ave.,43.654833,-79.387206,M5G 1R5,ON
4,Kyoto House Japanese Restaurant,Sushi Restaurant,143 Dundas St. West,43.655381,-79.385270,NaN,ON
5,Yueh Tung Chinese Restaurant,Chinese Restaurant,126 Elizabeth St.,43.655281,-79.385337,NaN,ON
6,Tundra Restaurant,Restaurant,145 Richmond Street West,43.650010,-79.385608,M5H 2L2,ON
7,Cali Restaurant,Vietnamese Restaurant,179 Dundas St. W.,43.655068,-79.386375,M5G,ON
8,Adega Restaurant,Restaurant,33 Elm St,43.657519,-79.383462,M5G 1H1,ON
9,Donatello Restaurant,Italian Restaurant,37 Elm St.,43.657489,-79.383605,M5G 1H1,ON


In [42]:
# drop rows with none values
clean_restaurant_dataframe3 = clean_restaurant_dataframe2.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
clean_restaurant_dataframe3

,name,categories,address,lat,lng,postalCode,state
0,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,43.654884,-79.385931,M5G 1R3,ON
2,Hong Shing Chinese Restaurant,Chinese Restaurant,195 Dundas St W,43.654925,-79.387089,M5G 1C7,ON
3,Wah Too Seafood Restaurant,Chinese Restaurant,56 Centre Ave.,43.654833,-79.387206,M5G 1R5,ON
6,Tundra Restaurant,Restaurant,145 Richmond Street West,43.650010,-79.385608,M5H 2L2,ON
7,Cali Restaurant,Vietnamese Restaurant,179 Dundas St. W.,43.655068,-79.386375,M5G,ON
8,Adega Restaurant,Restaurant,33 Elm St,43.657519,-79.383462,M5G 1H1,ON
9,Donatello Restaurant,Italian Restaurant,37 Elm St.,43.657489,-79.383605,M5G 1H1,ON
11,Anoush Restaurant,Restaurant,250 Dundas St W,43.654588,-79.389692,M5T 2Z5,ON
12,The Elm Tree Restaurant,Modern European Restaurant,43 Elm St,43.657397,-79.383761,M5G 1H1,ON
13,Wah Sing Seafood Restaurant,Chinese Restaurant,47 Baldwin Street,43.655936,-79.393783,M5T 1L1,ON


**Generating map to visualize churches, bakery shops and restaurants and clustering these together**

In [43]:
# create dataframe of churches, bakery shops and restaurants
church_neighbourhood_df = pd.concat([clean_church_dataframe3,clean_bakery_dataframe3,clean_restaurant_dataframe3], ignore_index=True,sort=True)
church_neighbourhood_df

,address,categories,lat,lng,name,postalCode,state
0,202 St. Patrick St.,Church,43.655329,-79.390672,Our Lady of Mount Carmel Catholic Church 嘉模聖母天主堂,NaN,ON
1,141 McCaul St.,Church,43.654551,-79.391359,St. Patrick's Church,NaN,ON
2,205 John St,Church,43.651568,-79.391267,St. George The Martyr Anglican Church,NaN,ON
3,23 Henry St.,Church,43.657248,-79.393972,Holy Trinity Russian Orthodox Church,NaN,ON
4,10 Trinity Sq,Church,43.654580,-79.381705,The Church of the Holy Trinity,NaN,ON
5,177 Beverley St,Church,43.655635,-79.394259,Chinese Presbyterian Church,NaN,ON
6,595 Bay St.,Bakery,43.656765,-79.383548,Bakery 18,NaN,ON
7,238 Queen Street West,Bakery,43.650096,-79.390378,Bakery & Kaffee Haus,NaN,ON
8,595 Bay St.,Bakery,43.656198,-79.382403,Kin-Kin Bakery & Bubble Tea,NaN,ON
9,595 Bay St,Bakery,43.656204,-79.383210,Paris Croissant Bakery Cafe,NaN,ON


In [44]:
# Generate map to visualize church neighborhood including bakery shops and restaurants
church_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(church_neighbourhood_df['lat'], church_neighbourhood_df['lng'], 
                                           church_neighbourhood_df['name'], church_neighbourhood_df['categories'],\
                                               church_neighbourhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(church_map)  
    
church_map

**Generating map to visualize hotels or event spaces, bakery shops and restaurants and clustering these together**

In [45]:
# create dataframe of hotels, bakery shops and restaurants
hotel_neighborhood_df = pd.concat([hotel_dataframe, clean_bakery_dataframe3,clean_restaurant_dataframe3], ignore_index=True)
hotel_neighborhood_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,address,categories,lat,lng,name,postalCode,state
0,123 Queen Street West,Hotel,43.650594,-79.384530,Sheraton Centre Toronto Hotel,M5H 2M9,ON
1,123 Test Drive,Hotel,43.658434,-79.387894,VFM Test Hotel,M2M 2M2,ON
2,108 Chestnut Street,Hotel,43.654608,-79.385942,DoubleTree by Hilton,M5G 1R3,ON
3,188 University Ave.,Hotel,43.649129,-79.386557,Shangri-La Toronto,M5H 0A3,ON
4,145 Richmond St W,Hotel,43.649946,-79.385479,Hilton,M5H 2L2,ON
5,525 Bay Street,Hotel,43.654728,-79.382422,Marriott Downtown at CF Toronto Eaton Centre,M5G 2L2,ON
6,123 Queen St. W,Event Space,43.651217,-79.383771,Grand Ballroom,M5H 2M9,ON
7,595 Bay St.,Bakery,43.656765,-79.383548,Bakery 18,NaN,ON
8,238 Queen Street West,Bakery,43.650096,-79.390378,Bakery & Kaffee Haus,NaN,ON
9,595 Bay St.,Bakery,43.656198,-79.382403,Kin-Kin Bakery & Bubble Tea,NaN,ON


In [47]:
# Generate map to visualize hotel neighbourhood including bakery shops and restaurants
hotel_map = folium.Map(location=[latitude, longitude], zoom_start=14)

for lat, lng, name, categories, address in zip(hotel_neighborhood_df['lat'], hotel_neighborhood_df['lng'], 
                                           hotel_neighborhood_df['name'], hotel_neighborhood_df['categories'],\
                                               hotel_neighborhood_df['address']):
    label = '{}, {}'.format(name, address)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(hotel_map)  
    
hotel_map